<a href="https://colab.research.google.com/github/AaryanAnand10/Pattern-Recognizing-/blob/main/DoubleTop_Bottom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema

In [3]:
class AdvancedPatternDetector:
    def __init__(self, polynomial_degree=17, window_delta=10, bottom_threshold=1.15,
                 top_threshold=0.85, pattern_threshold=0.12):

        self.polynomial_degree = polynomial_degree
        self.window_delta = window_delta
        self.bottom_threshold = bottom_threshold
        self.top_threshold = top_threshold
        self.pattern_threshold = pattern_threshold

In [6]:

    def fit_polynomial(self, x_data, y_data):
        """Fit polynomial to price data"""
        x = np.linspace(0, max(x_data), len(x_data))
        pol = np.polyfit(x_data, y_data, self.polynomial_degree)
        y_pol = np.polyval(pol, x)
        return x, y_pol
    
    def find_extrema(self, data):
        """Find local minima and maxima in polynomial fit"""
        # Find local minima and maxima using numpy diff
        min_max = np.diff(np.sign(np.diff(data))).nonzero()[0] + 1
        l_min = (np.diff(np.sign(np.diff(data))) > 0).nonzero()[0] + 1
        l_max = (np.diff(np.sign(np.diff(data))) < 0).nonzero()[0] + 1
        return l_min, l_max
    
    def analyze_price_windows(self, df, extrema, price_col):
        """Analyze price windows around extrema points"""
        dict_windows = {}
        df_len = len(df)
        
        for point in extrema:
            l_bound = max(0, point - self.window_delta)
            u_bound = min(df_len, point + self.window_delta + 1)
            x_range = range(l_bound, u_bound)
            
            price_window = []
            for x in x_range:
                if 0 <= x < df_len:
                    price_window.append(df[price_col].iloc[x])
            
            if price_window:
                avg_price = sum(price_window) / len(price_window)
                min_price = min(price_window)
                max_price = max(price_window)
                dict_windows[point] = {
                    'x_range': x_range,
                    'prices': price_window,
                    'avg_price': avg_price,
                    'min_price': min_price,
                    'max_price': max_price
                }
        
        return dict_windows
    
    def detect_patterns(self, df, high_col='High', low_col='Low', close_col='Close'):
        """
        Detect double top and bottom patterns using polynomial fitting approach.
        
        Parameters:
        df (pd.DataFrame): Price data
        high_col (str): Column name for high prices
        low_col (str): Column name for low prices
        close_col (str): Column name for close prices
        
        Returns:
        pd.DataFrame: Original data with pattern signals
        """
        result = df.copy()
        x_data = np.array(range(len(df)))
        
        # Initialize pattern columns
        result['double_top'] = 0
        result['double_bottom'] = 0
        result['pattern_strength'] = 0
        result['price_target'] = np.nan
        
        # Fit polynomials to high and low prices
        x, high_pol = self.fit_polynomial(x_data, df[high_col])
        x, low_pol = self.fit_polynomial(x_data, df[low_col])
        
        # Find extrema
        low_mins, low_maxs = self.find_extrema(low_pol)
        high_mins, high_maxs = self.find_extrema(high_pol)
        
        # Analyze windows around extrema
        bottom_windows = self.analyze_price_windows(df, low_mins, low_col)
        top_windows = self.analyze_price_windows(df, high_maxs, high_col)
        
        # Set thresholds
        global_min = df[low_col].min()
        global_max = df[high_col].max()
        bottom_thresh = global_min * self.bottom_threshold
        top_thresh = global_max * self.top_threshold
        
        # Detect double bottoms
        for i in range(len(low_mins) - 1):
            for j in range(i + 1, len(low_mins)):
                point1 = low_mins[i]
                point2 = low_mins[j]
                
                if point1 in bottom_windows and point2 in bottom_windows:
                    price1 = bottom_windows[point1]['avg_price']
                    price2 = bottom_windows[point2]['avg_price']
                    
                    # Check if prices are within threshold and below bottom threshold
                    if (abs(price1 - price2) / price1 < self.pattern_threshold and 
                        price1 < bottom_thresh and price2 < bottom_thresh):
                        
                        # Verify middle peak
                        middle_prices = df[high_col].iloc[point1:point2]
                        if len(middle_prices) > 0 and max(middle_prices) > max(price1, price2):
                            # Mark pattern
                            result.loc[point2, 'double_bottom'] = 1
                            result.loc[point2, 'pattern_strength'] = 1 - abs(price1 - price2) / price1
                            # Calculate target based on pattern height
                            pattern_height = max(middle_prices) - min(price1, price2)
                            result.loc[point2, 'price_target'] = df[close_col].iloc[point2] + pattern_height
        
        # Detect double tops
        for i in range(len(high_maxs) - 1):
            for j in range(i + 1, len(high_maxs)):
                point1 = high_maxs[i]
                point2 = high_maxs[j]
                
                if point1 in top_windows and point2 in top_windows:
                    price1 = top_windows[point1]['avg_price']
                    price2 = top_windows[point2]['avg_price']
                    
                    # Check if prices are within threshold and above top threshold
                    if (abs(price1 - price2) / price1 < self.pattern_threshold and 
                        price1 > top_thresh and price2 > top_thresh):
                        
                        # Verify middle trough
                        middle_prices = df[low_col].iloc[point1:point2]
                        if len(middle_prices) > 0 and min(middle_prices) < min(price1, price2):
                            # Mark pattern
                            result.loc[point2, 'double_top'] = 1
                            result.loc[point2, 'pattern_strength'] = 1 - abs(price1 - price2) / price1
                            # Calculate target based on pattern height
                            pattern_height = max(price1, price2) - min(middle_prices)
                            result.loc[point2, 'price_target'] = df[close_col].iloc[point2] - pattern_height
        
        return result, (x, high_pol, low_pol)

    def plot_patterns(self, df, fit_data, window_size=20):
        """
        Plot detected patterns with polynomial fits.
        
        Parameters:
        df: DataFrame with pattern signals
        fit_data: Tuple of (x, high_polynomial, low_polynomial)
        window_size: Number of periods to show around pattern
        """
        import matplotlib.pyplot as plt
        
        x, high_pol, low_pol = fit_data
        
        plt.figure(figsize=(15, 7))
        
        # Plot price data
        plt.plot(df.index, df['High'], 'o', markersize=1, color='grey', alpha=0.5, label='Price Data')
        plt.plot(df.index, df['Low'], 'o', markersize=1, color='grey', alpha=0.5)
        
        # Plot polynomial fits
        plt.plot(x, high_pol, '-', color='blue', alpha=0.5, label='High Fit')
        plt.plot(x, low_pol, '-', color='red', alpha=0.5, label='Low Fit')
        
        # Plot patterns
        for idx in df[df['double_top'] == 1].index:
            plt.axvspan(max(0, idx-window_size), min(len(df), idx+window_size), 
                       color='red', alpha=0.2)
            plt.plot(idx, df['High'].iloc[idx], 'rv', markersize=10, label='Double Top')
            
        for idx in df[df['double_bottom'] == 1].index:
            plt.axvspan(max(0, idx-window_size), min(len(df), idx+window_size), 
                       color='green', alpha=0.2)
            plt.plot(idx, df['Low'].iloc[idx], 'g^', markersize=10, label='Double Bottom')
        
        plt.title('Double Top/Bottom Patterns with Polynomial Fits')
        plt.legend()
        plt.grid(True)
        plt.show()